In [1]:
# Cell 1: Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Colab 셀에서 실행
%%writefile /content/drive/MyDrive/SemioticRAG/MinhwaOD/.gitignore
__pycache__/
*.pt
*.cache
data/images/
data/labels/
runs_minhwa/
artifacts/models/

Overwriting /content/drive/MyDrive/SemioticRAG/MinhwaOD/.gitignore


In [3]:
# 작업 디렉토리로 이동
# %cd /content/drive/MyDrive/SemioticRAG/MinhwaOD

# # 사용자 정보 설정 (Git에 필요한 경우)
# !git config --global user.email "smkim6927@gm.gist.ac.kr"
# !git config --global user.name "Sumin Kim"

# # Git 리포지토리에서 파일 가져오기 (만약 존재하지 않거나 업데이트가 필요한 경우)
# # 디렉토리가 비어있지 않은 경우 git clone 대신 git pull을 사용합니다.
# # 만약 'Minhwa_Object_Detection' 디렉토리가 존재하지 않는다면, 아래 명령어를 사용하세요:
# # !git clone https://github.com/smkim6927/Minhwa_Object_Detection.git .

# # 현재 디렉토리가 이미 Git 리포지토리로 초기화되어 있고, 원격 'origin'이 설정되어 있다고 가정합니다.
# # 원격 'origin'이 설정되어 있지 않다면 아래 주석 처리된 라인을 먼저 실행해야 합니다:
# # !git remote add origin https://github.com/smkim6927/Minhwa_Object_Detection.git

# # 원격 리포지토리에서 최신 변경사항을 가져옵니다.
# !git pull origin main


In [3]:
# Cell 0: Setup
!pip install ultralytics wandb

import ultralytics
ultralytics.checks()

Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40441MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.7/235.7 GB disk)


In [5]:
# !unzip -qq "/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/images.zip" \
#        -d "/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/images"

# !unzip -qq "/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels.zip" \
#        -d "/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels"

In [4]:
%cd /content/drive/MyDrive/SemioticRAG/MinhwaOD

/content/drive/MyDrive/SemioticRAG/MinhwaOD


In [5]:
import sys
sys.path.append('%cd /content/drive/MyDrive/SemioticRAG/MinhwaOD/src')

In [6]:
from pathlib import Path

dataset_root = "/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt"
artifacts_dir = str(Path(dataset_root) / "artifacts")

print(dataset_root)
print(artifacts_dir)

/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt
/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/artifacts


In [7]:
import sys
import os

# Ensure the 'src' directory is in the Python path
src_path = '/content/drive/MyDrive/SemioticRAG/MinhwaOD/src'
if src_path not in sys.path:
    sys.path.append(src_path)

# List of files to fix relative imports in
files_to_fix = ['pipeline.py', 'trainer.py', 'evaluator.py', 'predictor.py', 'exporter.py', 'wandb_utils.py']

for file_name in files_to_fix:
    file_path = os.path.join(src_path, file_name)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        new_content = content

        # Replace relative imports with absolute imports
        new_content = new_content.replace('from .config', 'from config')
        new_content = new_content.replace('from .trainer', 'from trainer')
        new_content = new_content.replace('from .evaluator', 'from evaluator')
        new_content = new_content.replace('from .predictor', 'from predictor')
        new_content = new_content.replace('from .exporter', 'from exporter')

        # Specific fix for SyntaxWarning in pipeline.py
        if file_name == 'pipeline.py':
            # Fix invalid escape sequence in docstring example path
            # Original contains: C:\...\KIDP_yolo_dt\artifacts
            # We escape the backslashes to avoid the SyntaxWarning
            new_content = new_content.replace(r'C:\...\KIDP_yolo_dt\artifacts', r'C:\\...\\KIDP_yolo_dt\\artifacts')

        # Removed the specific fix for wandb_utils.py that caused WandBManager to be unavailable.

        if content != new_content:
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)
            print(f"Fixed imports and potential syntax warnings in {file_name}")
    except FileNotFoundError:
        print(f"Warning: Could not find {file_name} to patch imports.")
    except Exception as e:
        print(f"Error patching {file_name}: {e}")

from config import TrainConfig
from pipeline import MinHwaDetectPipeline

In [8]:
!wandb login
# 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

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `export WANDB_API_KEY=<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>'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: smkim6927 (gist-infonet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
import os

label_base_dir = '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels'

print(f"Checking directory: {label_base_dir}")
if os.path.exists(label_base_dir):
    subdirs = os.listdir(label_base_dir)
    print("Subdirectories found:", subdirs)

    # Check if there is a nested 'labels' folder
    nested_labels = os.path.join(label_base_dir, 'labels')
    if os.path.exists(nested_labels) and os.path.isdir(nested_labels):
        print("\n[WARNING] Nested 'labels' folder detected!")
        print("Moving files from 'data/labels/labels' to 'data/labels'...")

        # Move command explanation: we will use shell command for efficiency
    else:
        print("\nStructure seems okay (no nested 'labels' folder found directly).")
else:
    print("Label base directory does not exist!")

Checking directory: /content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels
Subdirectories found: ['train', 'train.cache']

Structure seems okay (no nested 'labels' folder found directly).


In [10]:
import os
from pathlib import Path
import glob

# 1. Verify Label Existence for the first image
image_path_str = '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/images/train/0000049372P.JPG.jpg'
expected_label_path = image_path_str.replace('/images/', '/labels/').replace('.jpg', '.txt')

print(f"Image Path: {image_path_str}")
print(f"Expected Label Path: {expected_label_path}")

if os.path.exists(expected_label_path):
    print("[OK] Label file exists at expected path.")
else:
    print("[ERROR] Label file NOT found at expected path.")

    # Check for nested folder hypothesis
    nested_label_path = expected_label_path.replace('/data/labels/', '/data/labels/labels/')
    if os.path.exists(nested_label_path):
        print(f"[FOUND] But it is in a nested folder: {nested_label_path}")
        print("-> We need to move files from 'data/labels/labels' to 'data/labels'.")
    else:
        print("[FAIL] Label file not found even in nested folder.")

# 2. Clear YOLO cache files (important when dataset changes)
cache_pattern = '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/**/*.cache'
cache_files = glob.glob(cache_pattern, recursive=True)

if cache_files:
    print(f"\nFound {len(cache_files)} cache files. Deleting them to force re-scan...")
    for cache_file in cache_files:
        try:
            os.remove(cache_file)
            print(f"Deleted: {Path(cache_file).name}")
        except OSError as e:
            print(f"Error deleting {cache_file}: {e}")
else:
    print("\nNo cache files found.")

Image Path: /content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/images/train/0000049372P.JPG.jpg
Expected Label Path: /content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels/train/0000049372P.JPG.txt
[ERROR] Label file NOT found at expected path.
[FAIL] Label file not found even in nested folder.

Found 1 cache files. Deleting them to force re-scan...
Deleted: train.cache


In [11]:
import os

img_dir = '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/images/train'
label_dir = '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels/train'

print(f"--- Files in {img_dir} (First 5) ---")
if os.path.exists(img_dir):
    img_files = sorted(os.listdir(img_dir))
    for f in img_files[:5]:
        print(f)
else:
    print("Image directory not found!")

print(f"\n--- Files in {label_dir} (First 5) ---")
if os.path.exists(label_dir):
    label_files = sorted(os.listdir(label_dir))
    for f in label_files[:5]:
        print(f)
else:
    print("Label directory not found!")

# Check for extension match
if img_files and label_files:
    test_img = img_files[0]
    # Logic: YOLO replaces the last extension.
    # If file is 'a.JPG.jpg', it looks for 'a.JPG.txt'
    expected_label = os.path.splitext(test_img)[0] + '.txt'
    print(f"\n[Test Match]")
    print(f"Image: {test_img}")
    print(f"Expected Label: {expected_label}")
    if expected_label in label_files:
        print("-> MATCH FOUND in label directory.")
    else:
        print(f"-> MATCH NOT FOUND. (Closest might be: {label_files[0]}?)")

--- Files in /content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/images/train (First 5) ---
0000046116P.JPG.jpg
0000049110P.JPG.jpg
0000049111P.JPG.jpg
0000049112P.JPG.jpg
0000049114P.JPG.jpg

--- Files in /content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data/labels/train (First 5) ---
0000046116P.JPG.txt
0000049110P.JPG.txt
0000049111P.JPG.txt
0000049112P.JPG.txt
0000049114P.JPG.txt

[Test Match]
Image: 0000046116P.JPG.jpg
Expected Label: 0000046116P.JPG.txt
-> MATCH FOUND in label directory.


In [12]:
import yaml
from pathlib import Path
import os

base_path = Path('/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt')
yaml_path = base_path / 'data_ko_8020.yaml'
train_txt_path = base_path / 'train80.txt'

print(f"--- Inspecting {yaml_path.name} ---")
if yaml_path.exists():
    with open(yaml_path, 'r') as f:
        print(f.read())
else:
    print("YAML file not found!")

print(f"\n--- Inspecting first 5 lines of {train_txt_path.name} ---")
if train_txt_path.exists():
    with open(train_txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines[:5]:
            print(line.strip())

    # Check if first image exists
    if lines:
        first_img_path = lines[0].strip()
        if os.path.exists(first_img_path):
            print(f"\n[OK] First image found at: {first_img_path}")
        else:
            print(f"\n[ERROR] First image NOT found at: {first_img_path}")
else:
    print("Train txt file not found!")

--- Inspecting data_ko_8020.yaml ---
names:
  0: 범
  1: 까치
  2: 용
  3: 소나무
  4: 기러기
  5: 학
  6: 영지
  7: 봉황
  8: 오동나무
  9: 해
  10: 바다
  11: 바위
  12: 괴석
  13: 구름
  14: 꿩
  15: 동백
  16: 모란
  17: 목련
  18: 매화
  19: 꽃봉오리
  20: 닭
  21: 병아리
  22: 석류
  23: 복숭아
  24: 나비
  25: 토끼
  26: 소
  27: 거북
  28: 게
  29: 오징어
  30: 고양이
  31: 오리
  32: 화병
  33: 화분
  34: 버드나무
  35: 정자
  36: 사슴
  37: 산
  38: 물줄기
  39: 갈대
  40: 올챙이
  41: 잉어
  42: 쏘가리
  43: 숭어
  44: 메기
  45: 붕어
  46: 연꽃
  47: 연잎
  48: 연밥
  49: 잠자리
  50: 복숭아꽃
  51: 석류꽃
  52: 파랑새
  53: 팔가조
  54: 백로
  55: 원앙
  56: 제비
  57: 앵무새
  58: 물총새
  59: 국화
  60: 공작
  61: 계수나무
  62: 대나무
  63: 난초
  64: 방아깨비
  65: 아기새
  66: 송사리
  67: 참새
  68: 까마귀
  69: 복숭아나무
  70: 매화나무
  71: 달
  72: 단풍나무
  73: 호피
  74: 개
  75: 여의주
  76: 부엉이
  77: 비둘기
  78: 석류나무
  79: 목련나무
  80: 방울
  81: 술
  82: 해태
  83: 이름 모를 새
  84: 이름 모를 꽃
  85: 서기
  86: 파초
  87: 말
  88: 다리
  89: 배
  90: 집
  91: 사람
  92: 호랑이
  93: 동충하초
  94: 영지버섯
  95: 책
  96: 기린
  97: 해오라기
  98: 폭포
  99: 작약
  100: 곰
  101: 노루
 

In [13]:
import wandb.integration.ultralytics.callback as wandb_callback
import inspect

print("--- Debugging wandb.integration.ultralytics.callback ---")
print(f"Module: {wandb_callback}")

# Check if plot_detection_validation_results exists in the module
if hasattr(wandb_callback, 'plot_detection_validation_results'):
    print("[FOUND] plot_detection_validation_results exists in module.")
else:
    print("[MISSING] plot_detection_validation_results NOT found in module.")
    # List all attributes to see if it has a different name or if we need to look elsewhere
    print("Attributes:", dir(wandb_callback))

# Check if on_val_end exists
if hasattr(wandb_callback, 'on_val_end'):
    print("[FOUND] on_val_end exists.")
else:
    print("[MISSING] on_val_end NOT found.")

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


--- Debugging wandb.integration.ultralytics.callback ---
Module: <module 'wandb.integration.ultralytics.callback' from '/usr/local/lib/python3.12/dist-packages/wandb/integration/ultralytics/callback.py'>
[MISSING] plot_detection_validation_results NOT found in module.
Attributes: ['Callable', 'ClassificationPredictor', 'ClassificationTrainer', 'ClassificationValidator', 'DetectionPredictor', 'DetectionTrainer', 'DetectionValidator', 'PREDICTOR_TYPE', 'PosePredictor', 'PoseTrainer', 'PoseValidator', 'SAMPredictor', 'SegmentationPredictor', 'SegmentationTrainer', 'SegmentationValidator', 'TRAINER_TYPE', 'Union', 'VALIDATOR_TYPE', 'WandBUltralyticsCallback', 'YOLO', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'add_wandb_callback', 'annotations', 'copy', 'datetime', 'pickle', 'telemetry', 'torch', 'tqdm', 'ultralytics', 'version', 'wandb']
[MISSING] on_val_end NOT found.


In [14]:
%cd /content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt

# PYTHONPATH에 src 폴더를 추가하여 실행 시 모듈을 찾을 수 있게 함
!PYTHONPATH=/content/drive/MyDrive/SemioticRAG/MinhwaOD/src python /content/drive/MyDrive/SemioticRAG/MinhwaOD/main.py

/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt

[Before split sanity check]
{'yaml_path': '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data_ko.yaml',
 'path': '.',
 'train': 'train.txt',
 'val': None,
 'test': None,
 'names_count': 208}

[Split summary]
{'source_yaml': '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data_ko.yaml',
 'created_yaml': '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/data_ko_8020.yaml',
 'source_train_txt': '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/train.txt',
 'created_train_txt': '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/train80.txt',
 'created_test_txt': '/content/drive/MyDrive/SemioticRAG/MinhwaOD/preprocessing/KIDP_yolo_dt/test20.txt',
 'total': 279,
 'train_count': 223,
 'test_count': 56,
 'train_ratio': 0.8,
 'seed': 42,
 'make_list_items_absolute': True,
 'write_yaml_absolute_paths': True}
[Dat

In [15]:
import ultralytics, wandb
print("ultralytics", ultralytics.__version__)
print("wandb", wandb.__version__)

ultralytics 8.4.16
wandb 0.25.0
